In [1]:
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
deployment_name = os.getenv('DEPLOYMENT_NAME')

# A Few Shot Learning

In [2]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

😍,👏,👍


# Chain of Thought

Experiment results demonstrate Zero-shot-CoT using single prompt template, significantly outperform zero-shot LLM performance on diverse benchmark reasoning tasks.  Without any hand-crafted few-shot examples, e.g. increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

In [3]:
def call_openai(prompt, temperature=1, max_token=60):
    response = client.chat.completions.create(
        model=deployment_name,
        messages = prompt,
            max_tokens=max_token,
            temperature = temperature
    )
    return response.choices[0].message.content

In [4]:
# This prompt gets wrong answer - ZERO_SHOT Prompt

system_prompt = """Answer the following questions:"""
quiz = """The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning.
They sold 93 loaves in the morning and 39 loaves in the afternoon.
A grocery store returned 6 unsold loaves.
How many loaves of bread did they have left?"""

prompt = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

To determine how many loaves of bread the Beverly Hills Bakery had left, we need to account for the loaves sold and the loaves returned.

1. Start with the total number of loaves baked:
   \[
   200 \text{ loaves}
   \]

2. Subtract the number of loaves sold in the morning:
   \[
   200 - 93 = 107 \text{ loaves}
   \]

3. Subtract the number of loaves sold in the afternoon:
   \[
   107 - 39 = 68 \text{ loaves}
   \]

4. Add the number of loaves returned by the grocery store:
   \[
   68 + 6 = 74 \text{ loaves}
   \]

Therefore, the bakery had 74 loaves of bread left.


In [5]:
# Still wrong answer with few-shot learning

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": quiz}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

To find out how many loaves of bread the bakers at the Beverly Hills Bakery had left, we need to follow these steps:

1. Start with the total number of loaves baked on Monday morning: 200 loaves.
2. Subtract the number of loaves sold in the morning: 200 - 93 = 107 loaves.
3. Subtract the number of loaves sold in the afternoon: 107 - 39 = 68 loaves.
4. Add the number of loaves returned by the grocery store: 68 + 6 = 74 loaves.

So, the bakers had 74 loaves of bread left.


In [6]:
# With CoT, the answer is correct

cot_message = " Understand the sentences well and let’s think step by step."

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": quiz + cot_message}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

Let's break down the problem step by step to find out how many loaves of bread the Beverly Hills Bakery had left.

1. **Initial number of loaves baked:**
   - The bakers baked 200 loaves of bread on Monday morning.

2. **Number of loaves sold in the morning:**
   - They sold 93 loaves in the morning.

3. **Number of loaves sold in the afternoon:**
   - They sold 39 loaves in the afternoon.

4. **Number of loaves returned by the grocery store:**
   - A grocery store returned 6 unsold loaves.

First, let's calculate the total number of loaves sold during the day:
\[ 93 \text{ (morning sales)} + 39 \text{ (afternoon sales)} = 132 \text{ loaves sold} \]

Next, we subtract the total number of loaves sold from the initial number of loaves baked to find out how


In [7]:
# PROMPT_FEW_SHOT_CoT

prompt=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."""},
    {"role": "user", "content": quiz + cot_message}
]

response_msg = call_openai(prompt, temperature=0, max_token=200)

print(response_msg)

Let's break down the problem step by step:

1. The bakers baked 200 loaves of bread on Monday morning.
2. They sold 93 loaves in the morning.
3. They sold 39 loaves in the afternoon.

First, let's calculate the total number of loaves sold:
\[ 93 \text{ (morning sales)} + 39 \text{ (afternoon sales)} = 132 \text{ loaves sold} \]

Next, let's determine how many loaves were left after the sales:
\[ 200 \text{ (initial loaves)} - 132 \text{ (loaves sold)} = 68 \text{ loaves remaining} \]

Then, a grocery store returned 6 unsold loaves:
\[ 68 \text{ (remaining loaves)} + 6 \text{ (returned loaves)} = 74 \text{ loaves left} \]

So, the bakers had 74 loaves


# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)



Provide knowledge, turn knowledge question into reasoning. In general, more knowledge, better result.

3 Contributing factors:

(i) the quality of knowledge, 

(ii) the quantity of knowledge where the performance improves with more knowledge statements, and 

(iii) the strategy for integrating knowledge during inference

In [8]:
common_question = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""

prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

False. In golf, the objective is to have the lowest score possible. Players aim to complete the course in the fewest number of strokes.


In [9]:
common_question = """A tripod is a kind of easel
How many legs does an easel have?
"""
prompt = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

An easel typically has three legs, similar to a tripod. However, some easels, especially those designed for larger canvases or more stability, may have four legs. The traditional artist's easel usually has three legs, making it similar to a tripod in terms of structure.


## Check out follow 2 examples

In [10]:
# High confidence answer
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.
Explain and Answer: 
"""
prompt=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

No, the statement is incorrect. The objective of golf is not to get a higher point total than others. Instead, the goal is to complete a set of holes, typically 18, in the fewest number of strokes possible. Each stroke counts as one point, and the player with the lowest total number of strokes at the end of the round is the winner. Therefore, in golf, a lower score is better.


In [11]:
# Low confidence answer. 예문을 더 애매하게 바꿔야... 잘못된 결과가 나올것 같은데... 정상으로 나옴.
common_question = """Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: Golf is a precision club-and-ball sport in which competing players (or golfers) use several types of clubs to hit a ball into a series of holes on a course using the fewest number of strokes. A goal is highest rated with the lowest score calculated by adding up the total number of strokes taken on each hole. The player with the lowest score wins the game.
Explain and Answer: 
"""
prompt=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": common_question},
]

response_msg = call_openai(prompt, temperature=0, max_token=100)

print(response_msg)

No, part of golf is not trying to get a higher point total than others. 

In golf, the objective is to achieve the lowest score possible. Players use various types of clubs to hit a ball into a series of holes on a course, and the score is calculated by adding up the total number of strokes taken to complete each hole. The player with the fewest strokes, and thus the lowest score, wins the game. Therefore, unlike many other sports where a higher score indicates better performance
